In [1]:
import pandas
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import slack_alarm
import requests

# slack 알람 사용법
# slack_alarm.send_notification("CSV 파일 처리")

# 기능구현
1. text 불리기
2. user_data, detail 불리기
3. user_data, detail를 기반으로 contents, ERD_data, Erd_relationship, API 명세서 작성

In [2]:
text_dataset_list = df_columns_sort['text'].tolist()
domain_list = [
    "웹개발", "모바일앱", "PWA개발", "반응형웹", 
    "크로스플랫폼", "하이브리드앱", "웹어셈블리", "마이크로프론트엔드",
    "ERP시스템", "CRM시스템", "전자상거래", "재고관리시스템",
    "회계시스템", "프로젝트관리", "문서관리시스템", "워크플로우시스템",
    "핀텍", "디지털뱅킹", "결제시스템", "투자플랫폼",
    "AI머신러닝", "자연어처리", "컴퓨터비전", "데이터분석", 
    "빅데이터처리", "추천시스템",
    "헬스케어", "교육시스템", "스마트시티", "농업기술",
    "부동산시스템", "물류시스템", "여행예약시스템", "음식배달시스템",
    "게임개발", "스트리밍서비스", "소셜미디어",
    "블록체인개발", "IoT개발", "VR/AR개발"
]

NameError: name 'df_columns_sort' is not defined

In [10]:
import os
import re
import getpass
import matplotlib.pyplot as plt
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [11]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_texts(
    texts = text_dataset_list,
    embedding=embedding
)

/tmp/ipykernel_3165/824233482.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

Package                                  Version
---------------------------------------- ------------
accelerate                               0.29.3
aiobotocore                              2.7.0
aiofiles                                 24.1.0
aiohttp                                  3.9.3
aioitertools                             0.7.1
aiosignal                                1.2.0
alabaster                                0.7.12
altair                                   5.0.1
anaconda-anon-usage                      0.4.3
anaconda-catalogs                        0.2.0
anaconda-client                          1.12.3
anaconda-cloud-auth                      0.1.4
anaconda-navigator                       2.5.2
anaconda-project                         0.11.1
annotated-types                          0.7.0
anyio                                    4.2.0
appdirs                                  1.4.4
archspec                                 0.2.3
argon2-cffi                              21.3.

In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai
import pickle
import os

# 전역 변수로 모델과 데이터 저장
_model = None
_documents = []
_embeddings = []

def initialize_rag_model(model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
    """RAG 모델 초기화"""
    global _model
    if _model is None:
        _model = SentenceTransformer(model_name)
    return _model

def add_documents_to_rag(documents_list):
    """문서들을 RAG 시스템에 추가"""
    global _model, _documents, _embeddings
    
    if _model is None:
        initialize_rag_model()
    
    # 새 문서들 추가
    _documents.extend(documents_list)
    
    # 새 문서들의 임베딩 생성
    new_embeddings = _model.encode(documents_list)
    
    if len(_embeddings) == 0:
        _embeddings = new_embeddings
    else:
        _embeddings = np.vstack([_embeddings, new_embeddings])
    
    print(f"총 {len(_documents)}개 문서가 RAG 시스템에 등록되었습니다.")

def search_similar_documents(query, top_k=3):
    """쿼리와 유사한 문서 검색"""
    global _model, _documents, _embeddings
    
    if _model is None or len(_documents) == 0:
        return []
    
    # 쿼리 임베딩 생성
    query_embedding = _model.encode([query])
    
    # 코사인 유사도 계산
    similarities = cosine_similarity(query_embedding, _embeddings)[0]
    
    # 상위 k개 문서의 인덱스 찾기
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # 결과 구성
    results = []
    for idx in top_indices:
        results.append({
            'document': _documents[idx],
            'score': similarities[idx],
            'index': idx
        })
    
    return results

def keyword_based_selection(text_dataset_list, domain_list, top_k=6):
    """키워드 기반 선택 (기존 함수 유지)"""
    # 기존 로직 유지 - 여기서는 단순히 상위 k개 반환
    return text_dataset_list[:top_k] if len(text_dataset_list) >= top_k else text_dataset_list

def save_rag_data(filepath='rag_data.pkl'):
    """RAG 데이터를 파일로 저장"""
    global _documents, _embeddings
    
    data = {
        'documents': _documents,
        'embeddings': _embeddings
    }
    
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    
    print(f"RAG 데이터가 {filepath}에 저장되었습니다.")

def load_rag_data(filepath='rag_data.pkl'):
    """파일에서 RAG 데이터 로드"""
    global _documents, _embeddings
    
    if os.path.exists(filepath):
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        
        _documents = data['documents']
        _embeddings = data['embeddings']
        
        print(f"{len(_documents)}개 문서가 {filepath}에서 로드되었습니다.")
        return True
    else:
        print(f"{filepath} 파일을 찾을 수 없습니다.")
        return False

def textdummy_create_rag_enhanced(text_dataset_list, domain_list, model_name='gpt-4o-mini', use_rag=True, top_k=6):
    """RAG 기능이 추가된 텍스트 생성 함수"""
    
    if use_rag and len(_documents) > 0:
        # RAG를 사용하여 관련 문서 검색
        # 도메인 리스트를 쿼리로 사용
        query = " ".join(domain_list)
        relevant_docs = search_similar_documents(query, top_k)
        
        # 관련 문서들을 텍스트로 결합
        relevant_text = "\n\n---\n\n".join([doc['document'] for doc in relevant_docs])
        
        print(f"RAG 검색으로 {len(relevant_docs)}개의 관련 문서를 찾았습니다.")
        print("관련 문서 점수:", [f"{doc['score']:.3f}" for doc in relevant_docs])
        
    else:
        # 기존 키워드 기반 선택 사용
        relevant = keyword_based_selection(text_dataset_list, domain_list, top_k)
        relevant_text = "\n\n---\n\n".join(relevant)
        print(f"키워드 기반으로 {len(relevant)}개의 문서를 선택했습니다.")
    
    client = openai.OpenAI()

    # 최적화된 시스템 프롬프트
    OPTIMIZED_SYSTEM_PROMPT = """
    당신은 소프트웨어 프로젝트 기획 전문가입니다. 
    주어진 프로젝트 개요를 참고하여, 지정된 도메인에 맞는 새로운 프로젝트 개요를 생성하는 것이 목표입니다.

    **중요: 반드시 사용자가 지정한 도메인으로만 프로젝트를 생성해야 합니다.**

    **생성 규칙:**
    1. 지정된 도메인의 특성과 실제 개발 환경을 정확히 반영해야 합니다
    2. 기존 프로젝트의 구조와 문체를 참고하되, 완전히 새로운 프로젝트여야 합니다
    3. 실무에서 실제로 사용되는 도구, 방법론, 기술스택을 포함하세요
    4. 프로젝트의 목적, 주요 기능, 기술 스택이 명확히 드러나야 합니다
    5. 한 문단으로 구성된 연속적인 텍스트 형태로 작성하세요

    **출력 형식:**
    - 단일 문단 형태의 프로젝트 개요
    - 프로젝트명, 목적, 주요 기능, 기술 스택을 자연스럽게 포함
    - 예시와 같은 문체와 길이로 작성
    - 구체적이고 실현 가능한 프로젝트여야 함
    """

    enhanced_prompt = f"""
    아래 기존 프로젝트 개요를 참고하여, 제시된 도메인 카테고리 중 하나를 선택해 새로운 프로젝트 개요를 생성해주세요.

    선택 가능한 도메인 카테고리:
    {domain_list}

    참고할 프로젝트 개요 예시:
    {relevant_text}

    **요청사항:**
    1. 도메인 리스트에서 하나의 도메인을 무작위로 선택하세요
    2. 해당 도메인의 특성을 잘 반영한 새로운 프로젝트를 기획하세요
    3. 예시와 같은 문체와 구조로 한 문단으로 작성하세요
    4. 프로젝트명, 목적, 주요 기능, 기술 스택을 포함하세요
    5. 실제 개발 가능한 현실적인 프로젝트여야 합니다

    지금 새로운 프로젝트 개요 1개를 생성해주세요.
    """

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

# 사용 예시 함수
def setup_rag_system(documents_list):
    """RAG 시스템 설정"""
    print("RAG 시스템 초기화 중...")
    initialize_rag_model()
    add_documents_to_rag(documents_list)
    print("RAG 시스템 설정 완료!")

def test_rag_search(query, top_k=3):
    """RAG 검색 테스트"""
    results = search_similar_documents(query, top_k)
    
    print(f"\n'{query}' 검색 결과:")
    print("=" * 50)
    
    for i, result in enumerate(results, 1):
        print(f"\n[{i}] 유사도: {result['score']:.3f}")
        print(f"문서: {result['document'][:200]}...")
    
    return results

# 기존 함수 유지 (호환성을 위해)
def textdummy_create_rag(text_dataset_list, domain_list, model_name='gpt-4o-mini'):
    """기존 함수 (RAG 기능 없음)"""
    return textdummy_create_rag_enhanced(text_dataset_list, domain_list, model_name, use_rag=False)

# 다양한 도메인으로 여러 개 생성하는 함수 추가
def generate_multiple_projects(text_dataset_list, domain_list, count=5, model_name='gpt-4o-mini', use_rag=True):
    """여러 도메인으로 다양한 프로젝트 생성"""
    import random
    results = []
    
    for i in range(count):
        # 매번 다른 도메인 선택
        selected_domain = random.choice(domain_list)
        
        print(f"\n=== 프로젝트 {i+1}/{count} 생성 중 ===")
        
        result = textdummy_create_rag_enhanced(
            text_dataset_list=text_dataset_list,
            domain_list=domain_list,
            model_name=model_name,
            use_rag=use_rag,
            selected_domain=selected_domain
        )
        
        results.append({
            'domain': selected_domain,
            'project': result
        })
        
        print(f"✅ {selected_domain} 도메인 프로젝트 생성 완료\n")
    
    return results

In [16]:
# 랜덤 도메인 선택 (기존 방식)

result_lst = []

for i in range (0, 89) :
  result = textdummy_create_rag_enhanced(
      text_dataset_list,
      domain_list=[],
      use_rag=True
      # selected_domain 미지정시 자동으로 랜덤 선택
  )
  result_lst.append(result)
slack_alarm.send_notification("text_dataset 처리")
result_lst

키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.
키워드 기반으로 6개의 문서를 선택했습니다.


KeyboardInterrupt: 

In [17]:
result_lst

['이 앱은 청소년을 위한 안전한 소셜 네트워킹 플랫폼으로, 사용자들이 건강한 관계를 형성하고 서로의 경험을 공유할 수 있도록 돕는 것을 목표로 합니다. 사용자는 안전한 환경에서 친구를 만들고, 다양한 주제에 대한 토론을 통해 의견을 나눌 수 있으며, 실시간으로 서로의 게시물을 댓글로 소통할 수 있는 기능을 제공합니다. 또한, 부적절한 콘텐츠를 자동으로 필터링하고, 신고 기능을 통해 사용자 간의 안전을 보장하며, 보호자와의 연결 기능을 통해 부모가 자녀의 활동을 모니터링할 수 있도록 지원합니다. 이 플랫폼은 사용자 맞춤형 피드를 제공하여 관심사에 맞는 콘텐츠를 추천하고, 다양한 교육 자료와 정신 건강 관련 정보도 함께 제공하여 청소년들이 긍정적인 온라인 경험을 할 수 있도록 합니다. 서비스는 React Native를 기반으로 하여 모바일 환경에서도 최적화된 사용자 경험을 제공하며, Firebase를 통해 실시간 데이터 저장과 사용자 인증을 처리하여 안정성을 높였습니다.',
 '이 앱은 청소년을 위한 안전한 소셜 네트워킹 플랫폼으로, 사용자들이 서로의 관심사와 취미를 기반으로 연결될 수 있도록 돕습니다. 사용자는 프로필을 생성하고, 친구를 추가하여 소통하며, 안전한 환경에서 다양한 주제에 대한 게시물과 사진을 공유할 수 있습니다. 특히, 청소년의 안전을 고려하여 실시간 모니터링 시스템과 신고 기능이 탑재되어 있어, 부적절한 콘텐츠나 행동을 즉시 차단하고 관리자가 개입할 수 있도록 합니다. 또한, 사용자는 관심 있는 그룹에 가입하여 다양한 활동과 이벤트에 참여할 수 있으며, 친구와의 비공식적인 만남을 조율할 수 있는 기능도 포함되어 있습니다. 이 플랫폼은 청소년들이 건강한 관계를 형성하고, 안전하게 소통할 수 있는 공간을 제공하는 것을 목표로 하며, React Native와 Firebase를 활용하여 모바일 친화적인 사용자 경험을 구현하고, Node.js를 통해 서버 사이드 로직을 처리하여 안정성을 높였습니다.',
 '이 앱은 환경 보호를 위한 지역 커뮤니티 활동

In [6]:
dataset = df_columns_sort.copy()
text_dataset = dataset['text']
text_dataset

0     GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...
1     ‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...
2     이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...
3     이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...
4     이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...
                            ...                        
95    지역 축제의 혼잡도를 낮추고 운영 효율을 높이기 위해, 관람객의 이동 경로와 체류 ...
96    작물의 생장 조건을 분석하고 환경을 자동으로 조절하는 스마트팜 제어 시스템을 기획하...
97    청소년의 감정 상태를 분석하고 적절한 상담 자원을 연결해주는 정서 관리 플랫폼을 기...
98    가정에서 발생하는 음식물 쓰레기를 줄이기 위한 스마트 분리 수거 시스템을 기획하였다...
99    개인의 학습 집중도 패턴을 분석하여 최적의 공부 시간표를 자동으로 생성해주는 학습 ...
Name: text, Length: 100, dtype: object

In [18]:
initialize_rag_model()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\mir96\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mir96\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

c:\Users\mir96\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [19]:
# 1단계: RAG 시스템 설정 (한 번만)
existing_projects = text_dataset_list

setup_rag_system(existing_projects)

# 2단계: (선택사항) RAG 검색 테스트
# test_rag_search("웹개발 프로젝트", top_k=3)

# 3단계: 여러 프로젝트 생성
domain_list = [
    # 웹/모바일 플랫폼 (8개)
    "웹개발", "모바일앱", "PWA개발", "반응형웹", 
    "크로스플랫폼", "하이브리드앱", "웹어셈블리", "마이크로프론트엔드",
    
    # 비즈니스/기업 시스템 (8개)  
    "ERP시스템", "CRM시스템", "전자상거래", "재고관리시스템",
    "회계시스템", "프로젝트관리", "문서관리시스템", "워크플로우시스템",
    
    # 핀텍/금융 (4개)
    "핀텍", "디지털뱅킹", "결제시스템", "투자플랫폼",
    
    # AI/데이터 과학 (6개)
    "AI머신러닝", "자연어처리", "컴퓨터비전", "데이터분석", 
    "빅데이터처리", "추천시스템",
    
    # 특화 산업 도메인 (8개)
    "헬스케어", "교육시스템", "스마트시티", "농업기술",
    "부동산시스템", "물류시스템", "여행예약시스템", "음식배달시스템",
    
    # 엔터테인먼트/미디어 (3개)
    "게임개발", "스트리밍서비스", "소셜미디어",
    
    # 신기술/트렌드 (3개)
    "블록체인개발", "IoT개발", "VR/AR개발"
]

results = generate_multiple_projects(
    text_dataset_list=[],  # RAG 사용시 빈 리스트
    domain_list=domain_list,
    count=3,
    use_rag=True
)

# 결과 확인
for result in results:
    print(f"도메인: {result['domain']}")
    print(f"프로젝트: {result['project']}")
    print("-" * 50)

RAG 시스템 초기화 중...
총 100개 문서가 RAG 시스템에 등록되었습니다.
RAG 시스템 설정 완료!

=== 프로젝트 1/3 생성 중 ===


TypeError: textdummy_create_rag_enhanced() got an unexpected keyword argument 'selected_domain'